# How to Write the First Page of Your Paper with Python
An illustration using a siple GPT2 model. GPT3 has obviously reduced this process to n API call.

In [ ]:
!pip install transformers

In [ ]:
import re

import requests
import torch

import torch.nn as nn
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
# the following may take a few minutes for the download!
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large').to('cuda')

## Let's finetune the model on *Crime and Punishment*

In [ ]:
crime_n_punish = requests.get('http://www.gutenberg.org/files/2554/2554-0.txt').text
crime_n_punish = re.sub(r'[\n\r]+', ' ', crime_n_punish)
sentences = [s for s in re.findall(r'[^.?!]+[.?!]', crime_n_punish) if len(s) >= 20]

In [ ]:
model.train()

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=3e-5)

optimizer.zero_grad()

for epoch in range(4):

    running_loss = 0

    for i, sentence in enumerate(sentences[1000:1900]):

        tokenized = tokenizer.encode(sentence)
        index = torch.randint(2, len(tokenized) - 1, (1,)).item()
        tokenized = tokenized[:index]

        X = torch.Tensor(tokenized[:-1]).long().reshape(1, -1)
        X = X.to('cuda')

        y = torch.Tensor([tokenized[-1]]).long().to('cuda')

        output, _ = model(X)
        output = output[:, -1, :]

        loss = criterion(output, y)
        loss.backward()

        running_loss += loss

        if i % 2 == 0:
            optimizer.step()
            optimizer.zero_grad()
            print('EPOCH: {}, BATCH: {}, LOSS: {}'.format(epoch, i, running_loss / 2))
            running_loss = 0

# Now let's generate text based upon starting prompt

In [ ]:
start_of_paper = 'Ivan thinks a lot about '

In [ ]:
encoded_sequence = tokenizer.encode(start_of_paper)

In [ ]:
from pprint import pprint

model.eval()

with torch.no_grad():
    while len(encoded_sequence) < 450:
        predictions, _ = model(torch.Tensor([encoded_sequence]).long().to('cuda'))
        prediction = predictions[:, -1, :]

        # Ooooo, magic! (not really, but we won't have time to explain)
        topk = torch.topk(prediction[0], 10)
        values = F.softmax(topk.values, dim=0)
        indices = topk.indices
        indices_index = torch.multinomial(values, 1).item()
        next_word_index = indices[indices_index].item()
        
        encoded_sequence.append(next_word_index)
    
pprint(tokenizer.decode(encoded_sequence))

('Ivan thinks a lot about that in a day. He has seen it and he had thought he '
 'had lost something, he has lost the axe that is the axe that is in it and he '
 'was not looking at it, he was looking at it, looking for the axe to have '
 'returned from the bottom of a broken horn to one of his friends from out of '
 'the same ring, but the axe came back and his life over again was in order '
 'from the axe, he was alive and he was looking at the axe and the axe, it '
 'seemed to himself to have come to a point where he could not take his axe to '
 'do, it just seemed to himself to take a knife or to make one from one of his '
 'own in a knife-edge axe, but he knew that he could not have been thinking of '
 'that, he did not have a knife, he had not even a hammer, it was a straight '
 'edge, he could not have a straight edge of axe, he could not even make a '
 'straight edge with it in that, he could not make it by the knife and there '
 'was nothing to make a straight edge in that wit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun Mar  1 02:30:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    31W / 250W |   3997MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
torch.save(model, 'drive/My Drive/test.pt')

In [ ]:
import os

In [ ]:
os.listdir('drive')

['.shortcut-targets-by-id', 'Shared drives', 'My Drive', '.Trash']